In [1]:
import numpy as np
text = open('data/이광수_무정.txt', encoding='euc-kr').read()
print('말뭉치 크기:', len(text))

말뭉치 크기: 323659


In [2]:
# 60개 글자로 된 시퀀스를 추출합니다.
maxlen = 60
# 세 글자씩 건너 뛰면서 새로운 시퀀스를 샘플링합니다.
step = 3
# 추출한 시퀀스를 담을 리스트
sentences = []
# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 107867
고유한 글자: 1679
벡터화...


In [3]:
sentences[6]

'영어 교사 이형식은 오후 두시 사년급 영어 시간을 마치고 내려쪼이는 유월 볕에 땀을 흘리면서 안동 김장로의 '

In [4]:
next_chars[6]

'집'

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

In [6]:
from tensorflow.keras.optimizers import RMSprop
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
import random
import sys

random.seed(123)
start_index = random.randint(0, len(text) - maxlen - 1)

# 20 에포크 동안 모델을 훈련합니다
for epoch in range(1, 20):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 200개의 글자를 생성합니다
        for i in range(200):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
Train on 107867 samples
107867/107867 [==============================] - 36s 335us/sample - loss: 3.4100
--- 시드 텍스트: "지금껏 잊어버렸던 개가 입에 희끄무레한 무엇을 물고 따라온다. 영채는 반겨 그 개를 안았다. 그러나 그 개의"
------ 온도: 0.2
지금껏 잊어버렸던 개가 입에 희끄무레한 무엇을 물고 따라온다. 영채는 반겨 그 개를 안았다. 그러나 그 개의 ‘이 있는 사람이가 아니하고 그 사람이가 있는 사람이가 있는 사람이가 있다. 그러고 그 사람이가 그 사람이 있는 사람이가 있다. 그러고 그 사람이 있는 사람이가 아니하고 그 사람이 있는 사람이라. 그러고 형식의 생각이 되었다. 그러고 그 어디 형식의 생각이 하고 그 사람이 있는 사람이가 있는 사람이라. 그러나 그 사람이가 그러나 그 사람이 있는 사람이라.
------ 온도: 0.5
지금껏 잊어버렸던 개가 입에 희끄무레한 무엇을 물고 따라온다. 영채는 반겨 그 개를 안았다. 그러나 그 개의 모르는 무슨 듯이 하고 영채의 ‘영채는 이 있는 사람을 보고 영채의 정도 들어 사람이가 자기를 보고 얼굴이 아니하여 형식이가 우선은 형식을 하고 다만 그 사람이 있는 것이 보다. 그러고 영채의 말이 하고 부인에서 한 사람이 있다. 그러나 형식은 정한 힘이 있다. 그러나 형식의 ” 하고 남자는 태을 때에 형식의 사람이 있는 사람이가, 부인의 뜻을 하고 같다
------ 온도: 1.0
지금껏 잊어버렸던 개가 입에 희끄무레한 무엇을 물고 따라온다. 영채는 반겨 그 개를 안았다. 그러나 그 개의 청문슴이 된 지금 좋은 습니다을 무슨 뜻을 듣고 파는 기쁘을  있다는 여자거나 교육에 일하온 터를 깨고 斷면 형식의 만다. 자기 있는 녁보랐다. 고개를 듣들인가, 았다. 그러므로 알다니’ 하고 안 책을 보고 무먹 영채의 경도 동안 번 “친었다. 이 ‘마음’ 한 오직셨와 가기람도 위하는 거러운 말을 하던 형식의 듯이 식사가 모른다.
